In [1]:
import math
import numpy as np
import pandas as pd

In [2]:
pd.set_option("display.max_columns", None)

# Read in data

In [3]:
frames = pd.read_csv('../data/cleaned_train.csv')

C:\Users\ryanc\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (47) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
frames.shape

(682154, 58)

In [5]:
frames.head()

,GameId,PlayId,Team,X,Y,S,A,Dis,Orientation,Dir,NflId,DisplayName,JerseyNumber,Season,YardLine,Quarter,GameClock,PossessionTeam,Down,Distance,FieldPosition,HomeScoreBeforePlay,VisitorScoreBeforePlay,NflIdRusher,OffenseFormation,OffensePersonnel,DefendersInTheBox,DefensePersonnel,PlayDirection,TimeHandoff,TimeSnap,Yards,PlayerHeight,PlayerWeight,PlayerBirthDate,PlayerCollegeName,Position,HomeTeamAbbr,VisitorTeamAbbr,Week,Stadium,Location,StadiumType,Turf,GameWeather,Temperature,Humidity,WindSpeed,WindDirection,IsRusher,YardsToTouchdown,YardsToSafety,OffensivePlayer,PlayLeft,X_std,Y_std,Dir_rad,Dir_std
0,2017090700,20170907000118,away,73.91,34.84,4.0,1.13,0.40,81.99,177.18,496723,Eric Berry,29,2017,35,1,14:14:00,NE,3,2,NE,0,0,2543773,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,"2 DL, 3 LB, 6 DB",left,2017-09-08T00:44:06.000Z,2017-09-08T00:44:05.000Z,8,6-0,212,12/29/1988,Tennessee,SS,NE,KC,1,Gillette Stadium,"Foxborough, MA",Outdoor,Field Turf,Clear and warm,63.0,77.0,8.0,SW,0,65,35,0,1,46.09,18.46,4.761607,1.620015
1,2017090700,20170907000118,away,74.67,32.64,0.1,1.35,0.01,27.61,198.70,2495116,Allen Bailey,97,2017,35,1,14:14:00,NE,3,2,NE,0,0,2543773,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,"2 DL, 3 LB, 6 DB",left,2017-09-08T00:44:06.000Z,2017-09-08T00:44:05.000Z,8,6-3,288,03/25/1989,Miami,DE,NE,KC,1,Gillette Stadium,"Foxborough, MA",Outdoor,Field Turf,Clear and warm,63.0,77.0,8.0,SW,0,65,35,0,1,45.33,20.66,4.386012,1.244420
2,2017090700,20170907000118,away,74.00,33.20,3.1,0.59,0.31,3.01,202.73,2495493,Justin Houston,50,2017,35,1,14:14:00,NE,3,2,NE,0,0,2543773,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,"2 DL, 3 LB, 6 DB",left,2017-09-08T00:44:06.000Z,2017-09-08T00:44:05.000Z,8,6-3,270,01/21/1989,Georgia,DE,NE,KC,1,Gillette Stadium,"Foxborough, MA",Outdoor,Field Turf,Clear and warm,63.0,77.0,8.0,SW,0,65,35,0,1,46.00,20.10,4.315676,1.174083
3,2017090700,20170907000118,away,71.46,27.70,0.2,0.54,0.02,359.77,105.64,2506353,Derrick Johnson,56,2017,35,1,14:14:00,NE,3,2,NE,0,0,2543773,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,"2 DL, 3 LB, 6 DB",left,2017-09-08T00:44:06.000Z,2017-09-08T00:44:05.000Z,8,6-3,245,11/22/1982,Texas,ILB,NE,KC,1,Gillette Stadium,"Foxborough, MA",Outdoor,Field Turf,Clear and warm,63.0,77.0,8.0,SW,0,65,35,0,1,48.54,25.60,6.010216,2.868623
4,2017090700,20170907000118,away,69.32,35.42,1.6,2.43,0.16,12.63,164.31,2530794,Ron Parker,38,2017,35,1,14:14:00,NE,3,2,NE,0,0,2543773,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,"2 DL, 3 LB, 6 DB",left,2017-09-08T00:44:06.000Z,2017-09-08T00:44:05.000Z,8,6-0,206,08/17/1987,Newberry,FS,NE,KC,1,Gillette Stadium,"Foxborough, MA",Outdoor,Field Turf,Clear and warm,63.0,77.0,8.0,SW,0,65,35,0,1,50.68,17.88,4.986231,1.844638


# Create Features

Our play "blocks" will have the a structure where we relate offensive players by defensive players. I'm going to follow the same system as "The Zoo" did in their winning solution. They remove the RB from the block so they have a 11x10xn_features structure of features. Some of these features are simply relating offensive and defensive players directly, whereas others are defensive player vs RB features held constant across the 10 offensive players. For example, for each defensive player they calculated the differences in Sx, Sy, X_std and Y_std between the defenders and RB, and they also included the Sx and Sy of the defenders.

In [6]:
frames['Sx'] = frames['S']*frames['Dir_std'].apply(math.cos)
frames['Sy'] = frames['S']*frames['Dir_std'].apply(math.sin)

In [7]:
k = ['GameId','PlayId','X_std','Y_std','Dir_std','Sx','Sy']
o = frames.loc[(frames.OffensivePlayer == 1) & (frames.IsRusher == 0), k]
d = frames.loc[frames.OffensivePlayer == 0, k]

o.columns = ['GameId','PlayId','offense_X','offense_Y','offense_Dir','offense_Sx','offense_Sy']
d.columns = ['GameId','PlayId','defense_X','defense_Y','defense_Dir','defense_Sx','defense_Sy']

In [8]:
od = pd.merge(o, d, on=['GameId','PlayId'], how='outer')

In [9]:
od.head()

,GameId,PlayId,offense_X,offense_Y,offense_Dir,offense_Sx,offense_Sy,defense_X,defense_Y,defense_Dir,defense_Sx,defense_Sy
0,2017090700,20170907000118,44.18,35.74,6.036221,5.333125,-1.344537,46.09,18.46,1.620015,-0.196794,3.995156
1,2017090700,20170907000118,44.18,35.74,6.036221,5.333125,-1.344537,45.33,20.66,1.244420,0.032061,0.094721
2,2017090700,20170907000118,44.18,35.74,6.036221,5.333125,-1.344537,46.00,20.10,1.174083,1.197806,2.859241
3,2017090700,20170907000118,44.18,35.74,6.036221,5.333125,-1.344537,48.54,25.60,2.868623,-0.192595,0.053918
4,2017090700,20170907000118,44.18,35.74,6.036221,5.333125,-1.344537,50.68,17.88,1.844638,-0.432692,1.540382


In [10]:
# simple differences in X and Y coordinates
od['XOD'] = od['offense_X'] - od['defense_X']
od['YOD'] = od['offense_Y'] - od['defense_Y']
# Speed in X and Y directions
od['SxOD'] = od['offense_Sx'] - od['defense_Sx']
od['SyOD'] = od['offense_Sy'] - od['defense_Sy']

#### Defense vs RB features

In [11]:
rbs = frames[frames['IsRusher'] == 1][['GameId','PlayId','X_std','Y_std','Sx','Sy']]
rbs.columns = ['GameId','PlayId','rb_X','rb_Y','rb_Sx','rb_Sy']

In [12]:
od = pd.merge(od, rbs, on=['GameId','PlayId'], how='inner')

In [13]:
od['SxDRB'] = od['defense_Sx'] - od['rb_Sx']
od['SyDRB'] = od['defense_Sy'] - od['rb_Sy']
od['XDRB'] = od['defense_X'] - od['rb_X']
od['YDRB'] = od['defense_Y'] - od['rb_Y']

In [14]:
feats = ['XOD','YOD','SxOD','SyOD','SxDRB','SyDRB','XDRB','YDRB','defense_Sx','defense_Sy']
p = len(feats)

od_feats = od[feats].values.reshape(-1, 10, 11, p)
od_feats.shape

(31007, 10, 11, 10)

In [15]:
np.save(file='../data/offense_vs_defensive_features', arr=od_feats)